# TORCH 02. What is Pytorch?
- A replacement for Numpy to use the power of GPUs
- A deep learning research platform that provides maximum flexibility and speed

In [1]:
import numpy as np
import torch
print(torch.__version__)

1.4.0


## Getting Started

### Tensor

In [2]:
# Construct a 5x3 matrix, uninitialized
x = torch.empty(5, 3)
print(x)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [3]:
# Construct a randomly initialized matrix
x = torch.rand(5, 3)
print(x)

tensor([[0.2089, 0.1994, 0.3790],
        [0.1023, 0.4881, 0.0602],
        [0.4796, 0.9235, 0.9271],
        [0.9736, 0.0926, 0.3993],
        [0.1203, 0.8115, 0.4943]])


In [4]:
# Construct a matrix filled zeros and of dtype long
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [5]:
# Construct a tensor directly from data
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


In [6]:
# Create a tensor based on an existing tensor
# These methods will reuse properties of the input tensor,
# e.g. dtype, unless new values are provides by user
x = x.new_ones(5, 3, dtype=torch.double)    # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)  # override dtype!
print(x)                                    # result has the same size

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-3.3914,  0.0769, -0.0778],
        [-0.5808, -0.1656, -0.4057],
        [-1.4818, -1.5395, -1.1000],
        [-0.5421,  0.0707, -0.4674],
        [ 0.6956,  2.1925, -0.1657]])


In [7]:
print(x.size()) # `torch.Size` is in fact a tuple, so it supports all tuple operations

torch.Size([5, 3])


### Operations

In [9]:
# Addition: syntax 1
y = torch.rand(5, 3)
print(x + y)

# Addition: syntax 2
print(torch.add(x, y))

# Addition: providing an output tensor as argument
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

# Addition: in-place
y.add_(x)
print(y)

tensor([[-3.3656,  0.1205,  0.7102],
        [-0.4518,  0.0216, -0.1211],
        [-0.7046, -0.5462, -0.3677],
        [ 0.1752,  0.7133, -0.1449],
        [ 0.8272,  2.3320,  0.5012]])
tensor([[-3.3656,  0.1205,  0.7102],
        [-0.4518,  0.0216, -0.1211],
        [-0.7046, -0.5462, -0.3677],
        [ 0.1752,  0.7133, -0.1449],
        [ 0.8272,  2.3320,  0.5012]])
tensor([[-3.3656,  0.1205,  0.7102],
        [-0.4518,  0.0216, -0.1211],
        [-0.7046, -0.5462, -0.3677],
        [ 0.1752,  0.7133, -0.1449],
        [ 0.8272,  2.3320,  0.5012]])
tensor([[-3.3656,  0.1205,  0.7102],
        [-0.4518,  0.0216, -0.1211],
        [-0.7046, -0.5462, -0.3677],
        [ 0.1752,  0.7133, -0.1449],
        [ 0.8272,  2.3320,  0.5012]])


Note that: Any operation that mutates a tensor in-place is post-fixed with an `_`

For example, `x.copy_(y)`, `x.t_()`, will change `x`

In [10]:
# Resizing: if you want to resize/reshape tensor, you can use torch.view
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [11]:
# If you have a one element tensor, use `.items()` to get the value as a Python number
x = torch.randn(1)
print(x)
print(x.item())

tensor([1.0109])
1.010871171951294


Read later: 100+ Tensor operations, including transposing, indexing, slicing, mathematical operations, linear algebra, random numbers, etc., are described below

https://pytorch.org/docs/stable/torch.html

## Numpy Bridge
- Converting a Torch Tensor to a Numpy array and vice versa is a breeze(반대방향도 ㅇㅋ)

### Converting a Torch Tensor to a Numpy Array

In [12]:
a = torch.ones(5)
print('a :', a)
b = a.numpy()
print('b :', b)

a : tensor([1., 1., 1., 1., 1.])
b : [1. 1. 1. 1. 1.]


In [13]:
a.add_(1)
print('a :', a)
print('b :', b) # b도 변한다! 주소 참조 중

a : tensor([2., 2., 2., 2., 2.])
b : [2. 2. 2. 2. 2.]


### Converting Numpy Array to Torch Tensor

In [14]:
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print('a :', a)
print('b :', b)

a : [2. 2. 2. 2. 2.]
b : tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


## CUDA Tensors
- Tensors can be mobed onto any device using the `.to` method

In [15]:
# let us run this cell only if CUDA is available
# We will use `torch.device` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device('cuda')          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings `.to('cuda')`
    z = x + y
    print(z)
    print(z.to('cpu', torch.double))       # `.to` can also change dtype together!

tensor([2.0109], device='cuda:0')
tensor([2.0109], dtype=torch.float64)
